In [1]:
# Load your usual SpaCy model (one of SpaCy English models)
import spacy
nlp = spacy.load('en_core_web_sm') #spacy.load('en')

# load NeuralCoref and add it to the pipe of SpaCy's model
import neuralcoref
coref = neuralcoref.NeuralCoref(nlp.vocab)
nlp.add_pipe(coref, name='neuralcoref')

In [2]:
# import data
import pandas as pd
data_df = pd.read_csv('C:\\Users\\gcmar\\Desktop\\GIT_REPOS\\LAB\\Literature_summary\\Papers\\scopus_bark_ambrosia_beetles.csv')
data_df.columns

Index(['Unnamed: 0', 'Authors', 'Author(s) ID', 'Title', 'Year',
       'Source title', 'Volume', 'Issue', 'Art. No.', 'Page start', 'Page end',
       'Page count', 'Cited by', 'DOI', 'Link', 'Affiliations',
       'Authors with affiliations', 'Abstract', 'Author Keywords',
       'Index Keywords', 'Molecular Sequence Numbers', 'Chemicals/CAS',
       'Tradenames', 'Manufacturers', 'Funding Details', 'Funding Text 1',
       'Funding Text 2', 'Funding Text 3', 'Funding Text 4', 'Funding Text 5',
       'Funding Text 6', 'Funding Text 7', 'Funding Text 8', 'Funding Text 9',
       'Funding Text 10', 'References', 'Correspondence Address', 'Editors',
       'Sponsors', 'Publisher', 'Conference name', 'Conference date',
       'Conference location', 'Conference code', 'ISSN', 'ISBN', 'CODEN',
       'PubMed ID', 'Language of Original Document',
       'Abbreviated Source Title', 'Document Type', 'Publication Stage',
       'Open Access', 'Source', 'EID'],
      dtype='object')

In [3]:
def coref_res(text_series):
    coref_text_series = text_series.apply(lambda text : nlp(text)._.coref_resolved)
    return(coref_text_series)

In [4]:
coref_df = pd.DataFrame(coref_res(text_series=data_df["Abstract"][:10]))

In [5]:
coref_df

,Abstract
0,"Scolytinae species that, in high populations, ..."
1,Bamboo (Bambusa sp.) is a grass species with h...
2,Background: Biological invasions are responsib...
3,Background: Fungi associated with insects repr...
4,Background: Separation of biotic and abiotic i...
5,Microbial contamination of the domestic enviro...
6,Trunk internal decay is a common phenomenon in...
7,Wildfire is a widespread phenomenon on the ear...
8,The polyphagous shot hole borer (PSHB) Euwalla...
9,Innovation in environmental fields such as pla...


In [6]:
def call_wiki_api(item):
    try:
        url = f"https://www.wikidata.org/w/api.php?action=wbsearchentities&search={item}&language=en&format=json"
        data = requests.get(url).json()
        # Return the first id (Could upgrade this in the future)
        return data['search'][0]['id']
    except:
        return 'id-less'